In [2]:
import itertools
import pandas as pd
import cobra.flux_analysis
from cobra import Metabolite, Reaction, Model
import time
import numpy as np
from functools import partial
from src.mp_functions import combinations_subset, parallelize_dataframe, knockout_FBA, knockout_FBA_w_tasks

from functools import partial

Academic license - for non-commercial use only - expires 2022-10-03
Using license file c:\gurobi\gurobi.lic


NameError: name 'constrain_model' is not defined

In [ ]:
"""A mess of a document with different code cells.
Good to to use for any testing that involves the Recon3D model as it takes some time to load in."""

start_time = time.time()
model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/brain.xml'
model = cobra.io.read_sbml_model(model_file_path)

model_list = constrain_model(model, ALLMETSIN=True)
#Removes unused model.
model_list[2] = None
end_time = time.time()
print('Model load and preparation time: %.6f seconds' % (end_time - start_time))

In [22]:
def tasks_test(task_list: list, model_list: list, gene_ids: list) -> list:
    """Performs knockout FBA and checks tasks for the knockout. Returns a list of the results; objective value for
    general FBA and pass/fail for the tasks."""
    with model_list[0]:
        for gene_id in gene_ids:
            try:
                model_list[0].genes.get_by_id(gene_id).knock_out()
            except KeyError:
                return gene_id + ' not in model.'
        res = [model_list[0].slim_optimize()]

    for task in task_list:
        t_model = model_list[task[3]]

        with t_model:
            for subset in [task[0], task[1]]:
                for rx in subset:
                    if rx == 'ALLMETSIN':
                        # Adds boundary metabolites for other reactions when ALLMETSIN is used
                        for r in subset[1:]:
                            for m2 in r.metabolites:
                                for r2 in m2.reactions:
                                    if r2.boundary and r2.id != r.id:
                                        r2.add_metabolites({Metabolite(
                                                            m2.id[:-4] + 'x[x]',
                                                            formula=m2.formula,
                                                            name=' '.join(m2.name.split(' ')[:-1]) + ' [Boundary]',
                                                            compartment='x'): 1})
                        continue
                    t_model.add_reaction(rx)

            if task[2] != 'nan':
                t_model.add_reaction(task[2])

            for gene_id in gene_ids:
                t_model.genes.get_by_id(gene_id).knock_out()

            if t_model.slim_optimize(error_value='nan') == 'nan':
                res += [0]
            else:
                res += [1]


    return res

In [2]:
# Read and format task data
task_list = read_tasks('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/essential_tasks.tsv', model_list)

In [3]:
# Read test data
test_data = pd.read_csv('C:/Users/Sigve/Genome_Data/results/model_tests/test_data.csv')
test_data['pass/fail'] = test_data.values[:, 4:].tolist()
test_data = test_data[['phewas_code', 'gene_ids', 'solution', 'pass/fail']]
test_data['gene_ids'] = test_data['gene_ids'].apply(lambda x: x.split(','))
print(test_data.head())

# Reduce number of entries
#test_data = test_data.iloc[:10, :]

   phewas_code                            gene_ids   solution  \
0       290.11  [ENSG00000100030, ENSG00000197375] -80.729783   
1       290.11  [ENSG00000101577, ENSG00000197375] -80.729783   
2       290.11  [ENSG00000102780, ENSG00000197375] -80.729783   
3       290.11  [ENSG00000107798, ENSG00000197375] -80.729783   
4       290.11  [ENSG00000113448, ENSG00000197375] -80.729783   

                                           pass/fail  
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
2  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
3  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
4  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  


In [18]:
g = ['ENSG00000100030', 'ENSG00000197375']
%load_ext line_profiler
%lprun -f tasks_test tasks_test(task_list, model_list, g)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
*** KeyboardInterrupt exception caught in code being profiled.

In [25]:
gene_list = [[[g.id]] for g in model.genes]
print(len(gene_list))
gene_list = pd.DataFrame(gene_list, columns=['gene_ids'])
print(gene_list.head())

2451
            gene_ids
0  [ENSG00000000419]
1  [ENSG00000000938]
2  [ENSG00000001036]
3  [ENSG00000001084]
4  [ENSG00000001630]


In [39]:
start_time = time.time()

#test_data['results'] = test_data['gene_ids'].apply(partial(tasks_test, task_list, model_list))
test_results= parallelize_dataframe(gene_list, partial(combinations_subset, partial(knockout_FBA_w_tasks, task_list, model_list)), 16)


end_time = time.time()
print('FBA runtime: %.6f seconds' % (end_time - start_time))

FBA runtime: 5325.673097 seconds


In [40]:
copy = test_results.copy()

In [41]:
test_results['solution'] = test_results['results'].apply(lambda x: x[0])
test_results['results'] = test_results['results'].apply(lambda x: x[1:])
test_results['tasks_results'] = test_results['results']
test_results = test_results[['gene_ids', 'solution', 'tasks_results']]


In [7]:
# Converting MATLAB find all essential tasks
essential = pd.read_csv('C:/Users/Sigve/Genome_Data/results/model_tests/human1_essential.csv', index_col=0)

essential['pass/fail'] = essential.values.tolist()
essential = essential[['pass/fail']].reset_index()
print(essential.shape[0])
#essential['pass/fail'] = essential['pass/fail'].apply(lambda x: x[:-1] + [int(x[-1][0])])

#essential = essential[essential['pass/fail'].map(lambda x: any(task == 1 for task in x))]
essential['gene_ids'] = essential['Var1'].apply(lambda x: x)
essential = pd.DataFrame(essential[['gene_ids', 'pass/fail']])
essential['pass/fail'] = essential['pass/fail'].apply(lambda x: [1 if i==0 else 0 for i in x])
essential.reset_index(inplace=True, drop=True)
print(essential.head())
print(essential.shape[0])

3625
          gene_ids                                          pass/fail
0  ENSG00000000419  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
1  ENSG00000000938  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
2  ENSG00000001036  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
3  ENSG00000001084  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
4  ENSG00000001630  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
3625


In [17]:
model = cobra.io.load_matlab_model('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/Human-GEM.mat')

In [18]:
from cobra.flux_analysis import find_essential_genes
essential_genes = find_essential_genes(model, processes=12)
essential_genes = [gene.id for gene in essential_genes]

In [19]:
test = essential[essential['pass/fail'].map(lambda x: any([i==0 for i in x]))]['gene_ids'].tolist()
print(len(essential_genes))
print(len(test))
print(len(set(essential_genes).intersection(test)))

91
234
91


In [22]:
total = list(set(test + essential_genes))
print(len(model.genes))
total_non = [gene.id for gene in model.genes if gene.id not in total]
print(len(total))
print(len(total_non))
print(len(total+total_non))

pd.DataFrame(total, columns=['gene_ids']).to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/model_tests/essential_genes/human1_essential.tsv', sep='\t')
pd.DataFrame(total_non, columns=['gene_ids']).to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/model_tests/essential_genes/human1_non_essential.tsv', sep='\t')

3625
234
3391
3625
